In [1]:
!conda install pytorch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 pytorch-cuda=11.8 -c pytorch -c nvidia -y

Channels:
 - pytorch
 - nvidia
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='repo.anaconda.com', port=443): Read timed out. (read timeout=9.15)")': /pkgs/msys2/win-64/repodata.json.zst



In [2]:
import torch

In [3]:
#파이토치 GPU 사용 여부 확인
torch.cuda.is_available()

True

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as D
import numpy as np
import pickle
from tqdm import tqdm
import random
import math
from tqdm.notebook import tqdm

In [5]:
#word_to_index.pickle : 수어 스크립트 단어를 어떤 숫자로 변환 해야하는지 저장한 파일
with open('data/word_to_index.pickle', 'rb') as f:
    #수어 스크립트 단어를 어떤 숫자로 변환 해야하는지 저장한 파일 읽어서 word_to_index에 저장
    word_to_index = pickle.load(f)

In [6]:
word_to_index

{'<PAD>': 0,
 '<START>': 1,
 '<END>': 2,
 '6분': 3,
 '깊고': 4,
 '고흥군': 5,
 '되었을': 6,
 '빼고': 7,
 '20분': 8,
 '위경련': 9,
 '뒤집': 10,
 '벗고': 11,
 '털어': 12,
 '지탱': 13,
 '휴식': 14,
 '우리': 15,
 '용문산': 16,
 '35': 17,
 '1시': 18,
 '된': 19,
 '14': 20,
 '영등포구': 21,
 '바닥': 22,
 '지키시기': 23,
 '되었습니다': 24,
 '농사일': 25,
 '도크': 26,
 '식수': 27,
 '별': 28,
 '위험하므로': 29,
 '건물': 30,
 '세종': 31,
 '서': 32,
 '겼을': 33,
 '도내': 34,
 '물이': 35,
 '생긴': 36,
 '화악산': 37,
 '어려울': 38,
 '남부': 39,
 '14일': 40,
 '어깨': 41,
 '강변': 42,
 '그러한': 43,
 '때': 44,
 '05일': 45,
 '으로': 46,
 '실외': 47,
 '라디오': 48,
 '5월': 49,
 '여유': 50,
 '백운산': 51,
 '반': 52,
 '여주시': 53,
 '검은': 54,
 '나': 55,
 '완만': 56,
 '지날': 57,
 '시청': 58,
 '오후': 59,
 '북동쪽': 60,
 '15일': 61,
 '전라남도': 62,
 '야': 63,
 '건넌다': 64,
 '근육': 65,
 '뒤쪽': 66,
 '우천': 67,
 '청': 68,
 '낮게하고': 69,
 '로프': 70,
 '놀라서': 71,
 '경주': 72,
 '자': 73,
 '거북이': 74,
 '암벽': 75,
 '건너지': 76,
 '밀고': 77,
 '끼': 78,
 '달리는': 79,
 '사람': 80,
 '이나': 81,
 '물기': 82,
 '하천': 83,
 '맡기고': 84,
 '바랍니다': 85,
 '남남서': 86,
 '경기': 87,
 '

In [7]:
#len(word_to_index) : 저장된 단어의 개수
len(word_to_index)

1078

In [8]:
HID_DIM = 512
OUTPUT_DIM = len(word_to_index)
N_LAYER = 2 
DEC_DROPOUT = 0.5
EMB_DIM = 128
BATCH_SIZE = 32
N_EPOCH = 1000
CLIP = 1
learning_rate = 0.0001
model_save_path = 'data/'
save_model_name = 'model1.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

device: cuda:0


In [9]:
#np.load('data/X_data.npy') : keypoint가 저장죈 X_data.npy 읽어서 리턴
X_data = np.load('data/X_data.npy')

In [10]:
X_data.shape

(352, 240, 134)

In [11]:
X_data[0:3]

array([[[ 0.33139095, -1.2248869 ,  0.31375602, ...,  0.59203297,
         -0.07408668,  0.59665173],
        [ 0.33017254, -1.2249231 ,  0.31251508, ...,  0.593432  ,
         -0.07786492,  0.5957069 ],
        [ 0.32986647, -1.2254999 ,  0.3121397 , ...,  0.5932884 ,
         -0.07602021,  0.5910137 ],
        ...,
        [ 0.34666264, -1.2243315 ,  0.32900348, ...,  0.57456756,
         -0.10940002,  0.5609939 ],
        [ 0.34107053, -1.224629  ,  0.3319686 , ...,  0.5701985 ,
         -0.10562919,  0.5633026 ],
        [ 0.34480435, -1.2247169 ,  0.33569857, ...,  0.567693  ,
         -0.10210186,  0.5561903 ]],

       [[ 0.33823052, -1.2373524 ,  0.3206036 , ...,  0.5847284 ,
         -0.07379816,  0.5847284 ],
        [ 0.33649135, -1.2290341 ,  0.3270037 , ...,  0.58165026,
         -0.0718272 ,  0.59097505],
        [ 0.34267113, -1.2379397 ,  0.32487038, ...,  0.5861447 ,
         -0.07286945,  0.5861447 ],
        ...,
        [ 0.3576922 , -1.2186489 ,  0.31385562, ...,  

In [12]:
#np.load('data/y_data.npy) : 수어 스크립트가 저장된 y_data.npy 읽어서 리턴
y_data = np.load('data/y_data.npy')

In [13]:
y_data[0:3]

array([[  1, 103, 736, 720, 202, 954, 337, 226, 498, 916, 847, 900, 662,
        757, 123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  1, 505, 305, 818, 916, 954, 337, 226, 202, 498, 847, 900, 662,
        757, 123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0],
       [  1, 484, 448, 994, 954, 337, 226, 498, 498, 847, 900, 662, 757,
        123, 826, 840,  85,   2,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [14]:
y_data = torch.tensor(y_data)

In [15]:
y_data.shape

torch.Size([352, 30])

In [16]:
X_data = torch.tensor(X_data)

In [17]:
X_data.shape

torch.Size([352, 240, 134])

In [18]:
#D.TensorDataset(X_data,y_data): X_data와 y_data를 리턴할 객체 생성
train_dataset = D.TensorDataset(X_data, y_data)
train_dataset

In [19]:
#torch.utils.data.DataLoader( 
#                            train_dataset
#                            , batch_size=16 : 학습시 16행씩 리턴
#                            , shuffle=True : X_data 와 y_data의 행을 섞음
#                            , drop_last=False : 마지막 데이터의 행이 16개 미만일때도 학습 진행
#                            )
# : 학습시 X_data 와 y_data 를 리턴하는 객체 생성
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, drop_last=False)

In [20]:
train_dataloader

In [21]:
for index, (inp,ta) in enumerate(train_dataloader):
    print("index=",index)
    print("inp=",inp.shape)
    print("ta=",ta.shape)
    print("="*100)
    break

index= 0
inp= torch.Size([16, 240, 134])
ta= torch.Size([16, 30])


In [22]:
input_size = X_data.shape[2]

In [23]:
input_size

134

In [24]:
#인코더 연산을 클래스로 선언
class Encoder(nn.Module):
    def __init__(self, input_size, hid_dim, n_layer):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layer = n_layer
        self.gru = nn.GRU(
            input_size,
            hid_dim,
            n_layer,
            batch_first = True,
            bidirectional = True
        )

        self.fc = nn.Linear(
            hid_dim * 4,
            hid_dim
        )

    def forward(self, x):
        h0 = torch.zeros(self.n_layer*2, x.shape[0], self.hid_dim).cuda().float()
        encoder_output, encoder_hidden = self.gru(x,h0)    #<-- 수정

        encoder_hidden = torch.cat((
            encoder_hidden[-4,:,:],
            encoder_hidden[-3,:,:],
            encoder_hidden[-2,:,:],
            encoder_hidden[-1,:,:]
        ), dim = 1)

        encoder_hidden = torch.tanh(self.fc(encoder_hidden))
        return encoder_output, encoder_hidden

In [25]:
#Attention을 클래스로 구현

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()

        self.attn = nn.Linear((hid_dim * 2) + hid_dim, hid_dim)
        self.v = nn.Linear(hid_dim, 1, bias = False)

    def forward(self, hidden, encoder_output):

        batch_size = encoder_output.shape[0]
        src_len = encoder_output.shape[1]
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_output), dim = 2)))

        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim = 1)

In [26]:
#디코더 연산을 클래스로 구현
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layer, attention, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.attention = attention
        self.n_layer = n_layer
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.gru = nn.GRU(hid_dim*2+emb_dim,hid_dim)
        self.fc_out = nn.Linear((hid_dim*2)+hid_dim+emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, encoder_hidden, encoder_output):

        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))

        attention_output = self.attention(encoder_hidden, encoder_output)
        attention_output = attention_output.unsqueeze(1)

        weighted = torch.bmm(attention_output, encoder_output)

        weighted = weighted.permute(1,0,2)

        rnn_input = torch.cat((embedded, weighted), dim = 2)

        decoder_output, decoder_hidden = self.gru(rnn_input, encoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        decoder_output = decoder_output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = self.fc_out(torch.cat((decoder_output, weighted, embedded), dim = 1))

        return prediction,decoder_hidden.squeeze(0)

In [27]:
#SequenceToSequence 구현

class GRU_AT_Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    #딥러닝 학습시 호출되는 함수
    #src : 수어 keypoint
    #trg : 수어 스크립트 문장이 저장
    def forward(self, src, trg, teacher_forcing_ration = 0.5):
        #trg.shape[0] : trg에 저장된 문장의 개수 (16)
        batch_size = trg.shape[0]

        #trg.shape[1] : 수어 스크립트 단어의 개수 (30)
        trg_len = trg.shape[1]

        #decoder.output_dim : 수어 스크립트를 구성하는 단어의 개수 (162)
        trg_vocab_size = self.decoder.output_dim

        #torch.zeros(trg_len, batch_size, trg_vocab_size): 0으로 초기화된 배열 생성
                                                        # 배열 shape[trg_len(30), batch_size(16), trg_vocab_size(162)]
        output = torch.zeros(trg_len, batch_size, trg_vocab_size).cuda()

        #self.encoder(src) : 인코더 연산 실행
        encoder_output, hidden = self.encoder(src)

        #trg[:,0] :수어스크립트 trg 0번째 칸 리턴 <START>
        input = trg[:,0]

        for t in range(1, trg_len):
            #self.decoder(input, hidden, encoder_output) :디코더 연산 실행
            decoder_output, hidden = self.decoder(input, hidden, encoder_output)

            output [t] = decoder_output        #decoder_output: 디코더 연산 결과

            #random.random() < teacher_forcing_ration : 0~1 사이 난수를 생성해서 난수가 0.5 미만이면 난수 리턴
                                                     # 난수가 0.5이상이면 None 리턴

            teacher_force = random.random() < teacher_forcing_ration

            #decoder_output.argmax(1) : decoder_output에서 가장 확률이 높은 단어 대입
            top1 = decoder_output.argmax(1)

            #trg[:,t] if teacher_force : teacher_force 가 None이 아니면 수어스크립트가 저장된 trg[:,t] 리턴
            #  else top1:                teacher_force 가 None이면 decoder의 예측값 top1리턴
            input = trg[:,t] if teacher_force else top1

        return output

In [28]:
enc = Encoder(input_size, HID_DIM, N_LAYER).to(device)
enc

Encoder(
  (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=2048, out_features=512, bias=True)
)

In [29]:
att = Attention(HID_DIM).to(device)
att

Attention(
  (attn): Linear(in_features=1536, out_features=512, bias=True)
  (v): Linear(in_features=512, out_features=1, bias=False)
)

In [30]:
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYER, att, DEC_DROPOUT).to(device)
dec

Decoder(
  (attention): Attention(
    (attn): Linear(in_features=1536, out_features=512, bias=True)
    (v): Linear(in_features=512, out_features=1, bias=False)
  )
  (embedding): Embedding(1078, 128)
  (gru): GRU(1152, 512)
  (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [31]:
model = GRU_AT_Seq2Seq(enc,dec,device).to(device)
model

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [32]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [33]:
model.apply(init_weights)

GRU_AT_Seq2Seq(
  (encoder): Encoder(
    (gru): GRU(134, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=2048, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1078, 128)
    (gru): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=1078, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [34]:
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [35]:
def train(model, dataloader, OUTPUT_DIM,optimizer, criterion, clip):
    #학습시작
    model.train()

    epoch_loss = 0
    
    #dataloader : keypoint와 수어스크립트 단어를 리턴하는 객체
    #input: keypoint
    #target: 수어스크립트 단어
    for i, (input, target) in enumerate(dataloader):

        src = input #keypoint 대입
        trg = target #수어스크립트 단어 대입
        
        #torch.cuda.is_available() : GRU 사용 가능하면 True
        if torch.cuda.is_available():
            model.cuda() #model을 gru에 저장

            #src.cuda(): src를 GPU에 저장
            #.float(): 실수타입으로 변환
            src = src.cuda().float()

            #trg.cuda(): trg를 GPU에 저장
            #.long(): long타입으로 변환
            trg = trg.cuda().long()

        #0으로 초기화
        optimizer.zero_grad()
        #model 실행
        output = model(src, trg)

        #output[0] => <START>  입력의 예측결과 저장
        #output[1] : 수어 스크립트 저장

        #view(-1, OUTPUT_DIM): 배열 차원을 OUTPUT_DIM(162)열로 변환
                                # 행은 생략(-1)
        output = output[1:].view(-1, OUTPUT_DIM)
        #torch.transpose(trg,0,1): trg 배열 차원의 0번째(행) 1번째(열) 서로 바꾸기
        trg = torch.transpose(trg,0,1)

        #trg[1:].contiguous() : 배열에 저장된 요소들의 메모리 주소가 서로 인접하도록 설정
        #view(-1) :1 차원 배열로 변환
        trg = trg[1:].contiguous().view(-1)

        #criterion(output, trg) : output(예측한 수어스크립트) trg(실제 수어스크립트)의 차이를 계산
        loss = criterion(output, trg)

        #역전파 실행
        loss.backward()

        #output(예측한 수어스크립트) trg(실제 수어스크립트) 의 loss가 1이상이면 #loss를 clip(1)로 치환해서 역전파 실행
        torch.nn.utils.clip_grad_norm_(model.parameters(),clip)

        #역전파 결과로 매개변수 수정
        optimizer.step()

        epoch_loss += loss.item()
    return epoch_loss / len(dataloader)

In [36]:
#학습진행

best_loss = float('inf')

train_loss_ls = []  #<-- 수정

for epoch in tqdm(range(N_EPOCH)):
    #train 함수 호출
    train_loss = train(model, train_dataloader, OUTPUT_DIM, optimizer, criterion, CLIP)

    #train_loss를 리스트에 추가
    train_loss_ls.append(train_loss)

    #train_loss가 best_loss 미만
    if train_loss < best_loss:
        #train_loss를 대입
        best_loss = train_loss
        #학습모델 저장
        torch.save(model.state_dict(), f'{model_save_path}{save_model_name}')  #<-- 수정

    #cache 삭제
    torch.cuda.empty_cache()
    print(f'Epoch:{epoch + 1:02}')
    print(f'\t Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print("="*100)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch:01
	 Train Loss: 5.385 | Train PPL: 218.211
Epoch:02
	 Train Loss: 4.359 | Train PPL:  78.156
Epoch:03
	 Train Loss: 4.086 | Train PPL:  59.521
Epoch:04
	 Train Loss: 3.894 | Train PPL:  49.119
Epoch:05
	 Train Loss: 3.769 | Train PPL:  43.355
Epoch:06
	 Train Loss: 3.696 | Train PPL:  40.269
Epoch:07
	 Train Loss: 3.568 | Train PPL:  35.439
Epoch:08
	 Train Loss: 3.449 | Train PPL:  31.484
Epoch:09
	 Train Loss: 3.349 | Train PPL:  28.474
Epoch:10
	 Train Loss: 3.260 | Train PPL:  26.048
Epoch:11
	 Train Loss: 3.186 | Train PPL:  24.184
Epoch:12
	 Train Loss: 3.129 | Train PPL:  22.849
Epoch:13
	 Train Loss: 3.066 | Train PPL:  21.447
Epoch:14
	 Train Loss: 2.993 | Train PPL:  19.947
Epoch:15
	 Train Loss: 2.939 | Train PPL:  18.896
Epoch:16
	 Train Loss: 2.900 | Train PPL:  18.175
Epoch:17
	 Train Loss: 2.831 | Train PPL:  16.965
Epoch:18
	 Train Loss: 2.820 | Train PPL:  16.770
Epoch:19
	 Train Loss: 2.749 | Train PPL:  15.633
Epoch:20
	 Train Loss: 2.710 | Train PPL:  15.031
